In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn

In [3]:
#!gdown 1h86M8si2YT-aI4Zec1MeMP_mPYsLPy5F

In [5]:
df = pd.read_csv('HR-Employee-Attrition.csv')
df.drop(['EmployeeCount', 'EmployeeNumber', 'StandardHours', 'Over18'], axis=1, inplace=True)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [6]:
def unique_vals(col):

  if col.dtype == "object":

    print(f'{col.name}: {col.nunique()}')

df.apply(lambda col: unique_vals(col))





from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Create a label encoder object
le = LabelEncoder()

def label_encode(ser):

    if ser.dtype=="object" and ser.nunique() <= 2:
      print(ser.name)

      le.fit(ser)
      ser = le.transform(ser)

    return ser

df = df.apply(lambda col: label_encode(col))



# convert rest of categorical variable into dummy
df = pd.get_dummies(df, columns = ["BusinessTravel", "Department", "MaritalStatus"], drop_first = True)


target = df['Attrition'].copy()
df = df.drop(["Attrition"], axis = 1)
type(target)






# Since we have class imbalance (i.e. more employees with turnover=0 than turnover=1)
# let's use stratify=y to maintain the same ratio as in the training dataset when splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,
                                                    target,
                                                    test_size=0.25,
                                                    random_state=7,
                                                    stratify=target)

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)



import category_encoders as ce

ce_target = ce.TargetEncoder(cols = ['EducationField', 'JobRole'])
X_train = ce_target.fit_transform(X_train, y_train)
X_test = ce_target.transform(X_test)




#Upsampling using SMOTE

from imblearn.over_sampling import SMOTE
from collections import Counter

smt = SMOTE()
X_sm, y_sm = smt.fit_resample(X_train, y_train)

print('Resampled dataset shape {}'.format(Counter(y_sm)))

Attrition: 2
BusinessTravel: 3
Department: 3
EducationField: 6
Gender: 2
JobRole: 9
MaritalStatus: 3
OverTime: 2
Attrition
Gender
OverTime
Number transactions X_train dataset:  (1102, 33)
Number transactions y_train dataset:  (1102,)
Number transactions X_test dataset:  (368, 33)
Number transactions y_test dataset:  (368,)
Resampled dataset shape Counter({0: 924, 1: 924})


In [7]:
from sklearn.ensemble import GradientBoostingClassifier

In [9]:
gbc = GradientBoostingClassifier(n_estimators = 150)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=150)

In [10]:
gbc.score(X_train, y_train)

0.9773139745916516

In [11]:
gbc.score(X_test, y_test)

0.875